In [ ]:
import socket
import sys
import threading

# Project modules
import import_ipynb
import http_util


class WebProxy():

    def __init__(self, proxy_host, proxy_port):
        self.proxy_host = proxy_host
        self.proxy_port = proxy_port
        self.proxy_backlog = 1
        self.cache = {}
        self.start()

    def start(self):
        proxy_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        proxy_sock.bind((self.proxy_host, self.proxy_port))
        proxy_sock.listen(self.proxy_backlog)
        # Wait for client connection
        while True:
            conn, addr = proxy_sock.accept()
            print ('Client has connected', addr)
            thread = threading.Thread(target = self.serve_content, args = (conn, addr))
            thread.start()
        
    def serve_content(self, conn, addr):

        # Receive binary request from client
        bin_req = conn.recv(4096)
        try:
            str_req = bin_req.decode('utf-8')
            print(str_req)
        except ValueError as e:
            print ("Unable to decode request, not utf-8", e)
            conn.close()
            return

        # Extract host and path
        hostname = http_util.get_http_field(str_req, 'Host: ', '\r\n')
        pathname = http_util.get_http_field(str_req, 'GET ', ' HTTP/1.1')
        if hostname == -1 or pathname == -1:
            print ("Cannot determine host")
            client_conn.close()
            return
        elif pathname[0] != '/':
            [hostname, pathname] = http_util.parse_url(pathname)
        str_req = http_util.create_http_req(hostname, pathname)

        url = hostname + pathname
        #Checks if url is in the cache and if is, save to keyVal
        if(url in self.cache):
            print("url is in cache")
            cache_temp = self.cache[url]
            add_date = cache_temp[0]
            new_str_req = http_util.add_http_field(str_req, "If-Modified-Since", str(add_date))
            str_req = new_str_req
            #Check the cache contents
        # Open connection to host and send binary request
        bin_req = str_req.encode('utf-8')
        try:
            web_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            web_sock.connect((hostname, 80))
            print ("Sending request to web server: ", str_req)
            web_sock.sendall(bin_req)
        except OSError as e:
            print ("Unable to open web socket: ", e)
            if web_sock:
                web_sock.close()
            conn.close()
            return

        # Wait for response from web server
        bin_reply = b''
        while True:
            more = web_sock.recv(4096)
            if not more:
                 break
            bin_reply += more
        try:
            decoded = bin_reply.decode('utf-8')
            try:
                date = http_util.get_http_field(decoded, "Last-Modified: ","\r\n")
            except:
                date = http_util.get_http_field(decoded, "Date: ", "\r\n")
        except ValueError as e:
            print("Unable to Decode Url Object", e)            
        # Proxy now Checks Server Response

        # if there is no modification to dict
        if(http_util.get_http_field(decoded, "HTTP/1.1 ", "\r\n") == "304 Not Modified"):
            print("Server sent back 304 Not Modified.")
            #Update cache to not modified...
            conn.sendall(self.cache[url][1])
            print("Came from the Cache, 304 Not Modified")
        # if server returns 200 OK
        elif(http_util.get_http_field(decoded, "HTTP/1.1 ", "\r\n") == "200 OK"):
            self.cache[url] = [date,bin_reply]
            print('Proxy received from server (showing 1st 300 bytes): ', bin_reply[:300])
            conn.sendall(bin_reply)
        # if there is some kind of error
        else: #if there is a problem
            print("Error with Request, Closing connection.")
            conn.close() 
        # Close connection to client
        conn.close()
# Main
proxy_host = 'localhost'
proxy_port = 50007
web_proxy = WebProxy(proxy_host, proxy_port)


['C:\\Users\\Abinaya_UB\\anaconda3\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Abinaya_UB\\AppData\\Roaming\\jupyter\\runtime\\kernel-e82b3060-b5ef-4a9a-bdfb-a2f0aa965384.json'] 3
Client has connected ('127.0.0.1', 51880)
GET / HTTP/1.1
Host: cattheory.com
Accept-charset: utf-8
Connection: close


Sending request to web server:  GET / HTTP/1.1
Host: cattheory.com
Accept-charset: utf-8
Connection: close


Proxy received from server (showing 1st 300 bytes):  b'HTTP/1.1 200 OK\r\nServer: GitHub.com\r\nDate: Thu, 18 Nov 2021 16:30:46 GMT\r\nContent-Type: text/html; charset=utf-8\r\nContent-Length: 9575\r\nVary: Accept-Encoding\r\nLast-Modified: Tue, 11 May 2021 22:43:33 GMT\r\nVary: Accept-Encoding\r\nAccess-Control-Allow-Origin: *\r\nETag: "609b0895-2567"\r\nexpires: Thu, 18 '
Client has connected ('127.0.0.1', 51900)
GET / HTTP/1.1
Host: cattheory.com
Accept-charset: utf-8
Connection: close


url is in cache
Sending request to web server:  GET / HTTP/1.1
Host: cattheor

In [ ]:
import socket
import sys
import threading

# Project modules
import import_ipynb
import http_util


class WebProxy():

    def __init__(self, proxy_host, proxy_port):
        self.proxy_host = proxy_host
        self.proxy_port = proxy_port
        self.proxy_backlog = 1
        self.cache = {}
        self.start()

    def start(self):

        # Initialize server socket on which to listen for connections
        try:
            proxy_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            proxy_sock.bind((self.proxy_host, self.proxy_port))
            proxy_sock.listen(self.proxy_backlog)
        except OSError as e:
            print ("Unable to open proxy socket: ", e)
            if proxy_sock:
                proxy_sock.close()
            sys.exit(1)

        # Wait for client connection
        while True:
            conn, addr = proxy_sock.accept()
            print ('Client has connected', addr)
            thread = threading.Thread(target = self.serve_content, args = (conn, addr))
            thread.start()
        
    def serve_content(self, conn, addr):

        # Receive binary request from client
        bin_req = conn.recv(4096)
        try:
            str_req = bin_req.decode('utf-8')
            print(str_req)
        except ValueError as e:
            print ("Unable to decode request, not utf-8", e)
            conn.close()
            return

        # Extract host and path
        hostname = http_util.get_http_field(str_req, 'Host: ', '\r\n')
        pathname = http_util.get_http_field(str_req, 'GET ', ' HTTP/1.1')
        if hostname == -1 or pathname == -1:
            print ("Cannot determine host")
            client_conn.close()
            return
        elif pathname[0] != '/':
            [hostname, pathname] = http_util.parse_url(pathname)
        str_req = http_util.create_http_req(hostname, pathname)

        url = hostname + pathname
        #-----------------------------My Code----------------------------------------
        #Checks if url is in the cache and if is, save to keyVal
        if(url in self.cache):
            print("url is in cache")
            cache_temp = self.cache[url]
            add_date = cache_temp[0]
            new_str_req = http_util.add_http_field(str_req, "If-Modified-Since", str(add_date))
            str_req = new_str_req
            #Check the cache contents		
        # Open connection to host and send binary request
        bin_req = str_req.encode('utf-8')
        try:
            web_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            web_sock.connect((hostname, 80))
            print ("Sending request to web server: ", str_req)
            web_sock.sendall(bin_req)
        except OSError as e:
            print ("Unable to open web socket: ", e)
            if web_sock:
                web_sock.close()
            conn.close()
            return

        # Wait for response from web server
        bin_reply = b''
        while True:
            more = web_sock.recv(4096)
            if not more:
                 break
            bin_reply += more
        #-------Decode here----------
        try:
            decoded = bin_reply.decode('utf-8')
            try:
                date = http_util.get_http_field(decoded, "Last-Modified: ","\r\n")
            except:
                date = http_util.get_http_field(decoded, "Date: ", "\r\n")
        except ValueError as e:
            print("Unable to Decode Url Object", e)            
        #Proxy now Checks Server Response

        #-----if there is no modification to dict---
        if(http_util.get_http_field(decoded, "HTTP/1.1 ", "\r\n") == "304 Not Modified"):
            print("Server sent back 304 Not Modified.")
            #Update cache to not modified...
            conn.sendall(self.cache[url][1])
            print("Came from the Cache, 304 Not Modified")
        #---------if server returns 200 OK--------	
        elif(http_util.get_http_field(decoded, "HTTP/1.1 ", "\r\n") == "200 OK"):
            self.cache[url] = [date,bin_reply]
            print('Proxy received from server (showing 1st 300 bytes): ', bin_reply[:300])
            conn.sendall(bin_reply)
        #----------if there is some kind of error----
        else: #if there is a problem
            print("Error with Request, Closing connection.")
            conn.close() 
        #---------Close connection to client------
        conn.close()

def main():

    proxy_host = 'localhost'
    proxy_port = 50007

    '''if len(sys.argv) > 1:
        proxy_host = sys.argv[1]
        proxy_port = int(sys.argv[2])'''

    web_proxy = WebProxy(proxy_host, proxy_port)

if __name__ == '__main__':

    main()

importing Jupyter notebook from http_util.ipynb
importing Jupyter notebook from http_constants.ipynb
Client has connected ('127.0.0.1', 51314)
GET / HTTP/1.1
Host: info.cern.ch
Accept-charset: utf-8
Connection: close


Sending request to web server:  GET / HTTP/1.1
Host: info.cern.ch
Accept-charset: utf-8
Connection: close


Proxy received from server (showing 1st 300 bytes):  b'HTTP/1.1 200 OK\r\nDate: Thu, 18 Nov 2021 17:48:18 GMT\r\nServer: Apache\r\nLast-Modified: Wed, 05 Feb 2014 16:00:31 GMT\r\nETag: "286-4f1aadb3105c0"\r\nAccept-Ranges: bytes\r\nContent-Length: 646\r\nConnection: close\r\nContent-Type: text/html\r\n\r\n<html><head></head><body><header>\n<title>http://info.cern.ch</title>'
Client has connected ('127.0.0.1', 51316)
GET / HTTP/1.1
Host: info.cern.ch
Accept-charset: utf-8
Connection: close


url is in cache
Sending request to web server:  GET / HTTP/1.1
Host: info.cern.ch
Accept-charset: utf-8
Connection: close
If-Modified-Since: Wed, 05 Feb 2014 16:00:31 GMT


HTTP